In [5]:
pip install PyPDF2

In [10]:
import PyPDF2
import re
from pathlib import Path

def read_pdf(file_path):
    """Read PDF and return a list of tuples containing page number and text."""
    try:
        with open(file_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            pdf_content_by_page = []
            for page_num, page in enumerate(reader.pages):
                page_text = page.extract_text() or "[No text found on this page]"
                pdf_content_by_page.append((page_num + 1, page_text))
            return pdf_content_by_page
    except FileNotFoundError:
        return "Error: File not found. Please check the file path."
    except Exception as e:
        return f"An error occurred: {e}"

def highlight_query(snippet, query):
    """Highlight the query in the snippet."""
    highlighted = re.sub(f"({re.escape(query)})", r"\033[1;32m\1\033[0m", snippet, flags=re.IGNORECASE)
    return highlighted

def search_query(content_by_page, query):
    """Search for the query in the PDF and return snippets with page numbers."""
    results = []
    for page_num, page_text in content_by_page:
        matches = [m.start() for m in re.finditer(re.escape(query), page_text, re.IGNORECASE)]
        for start_idx in matches:
            snippet = page_text[max(0, start_idx - 100):start_idx + 400]
            highlighted_snippet = highlight_query(snippet, query)
            results.append((page_num, highlighted_snippet))
    return results

def chat_with_pdf(content_by_page):
    """Chatbot interface for querying the PDF."""
    print("Chat with your PDF! Type 'exit' to quit.")
    while True:
        user_query = input("\nYou: ").strip()
        if user_query.lower() == 'exit':
            print("Chat ended. Goodbye!")
            break
        elif user_query:
            results = search_query(content_by_page, user_query)
            if results:
                for i, (page_num, snippet) in enumerate(results, 1):
                    print(f"\nResult {i} (Page {page_num}):\n{snippet}...\n")
                    if i % 3 == 0:  # Pause after every 3 results
                        cont = input("Type 'more' to see more results or press Enter to stop: ").strip().lower()
                        if cont != 'more':
                            break
            else:
                print("PDF Bot: Sorry, I couldn't find anything related to your query in the document.")
        else:
            print("PDF Bot: Please enter a valid query.")

if __name__ == "__main__":
    file_path = input("Enter the path to the PDF file: ").strip()
    if not Path(file_path).exists():
        print("Error: File not found. Please check the file path.")
    else:
        pdf_content_by_page = read_pdf(file_path)
        if isinstance(pdf_content_by_page, str) and "Error" in pdf_content_by_page:
            print(pdf_content_by_page)
        else:
            chat_with_pdf(pdf_content_by_page)

Chat with your PDF! Type 'exit' to quit.
You: probability
 
PDF Bot: Here's what I found: 
PROBABILITY
2024-25

--- Page 2 ---
290 MATHEMA TICS
14.1.1   Occurrence of an event   Consider the experiment of throwing a die. Let E
denotes the event “ a number  less than 4 appears”. If actually ‘1’ had appeared on the
die then we say that event E has occurred. As a matter of fact if outcomes are 2 or 3,
we say that event E has occurred
Thus, the event E of a sample space S is said to have occurred if the outcome
ωof the experiment is such that ω∈ E. If the outcome ω is such that ω ∉ E, we ...

You: Range
 
PDF Bot: Here's what I found: 
range is the interval [0,1]
satisfying the following axioms
(i)For any event E,  P  (E) ≥0 (ii) P (S) = 1
(iii) If E and F are mutually exclusive events, then P(E ∪ F) = P(E) + P(F).
It follows from (iii) that P( φ) = 0. To prove this, we take F = φ and note that E and φ
are disjoint events. Therefore, from axiom (iii), we get
P (E ∪ φ) = P (E) + P ( φ)or P(